### Part One - Download the file, store the data in dataframe and wrangle the data

###### Installing the modules

In [1]:
! pip install lxml html5lib beautifulsoup4 pgeocode

###### Import Libraries

In [31]:
import pandas as pd
import pgeocode
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import numpy as np

###### Download the file

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url)

###### Convert the file in dataframe and ensure that the dataframe consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
wp_df=pd.DataFrame(df[0])
wp_df.columns

Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')

###### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [5]:
index_names=wp_df[wp_df['Borough']=='Not assigned'].index
wp_df.drop(index_names,inplace=True)

###### Postal code in the dataframe is unique, if two neighbourhoods are associated with single postal code there is only one row available in the dataframe for the postal code with the neighbourhood seperated with comma

In [6]:
wp_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


###### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [7]:
wp_df['Neighbourhood'] = wp_df.apply(lambda x: x['Borough'] if x['Neighbourhood']=='Not assigned' else x['Neighbourhood'], axis=1)

###### Use of .shape to get the total number of rows in the dataframe

In [8]:
wp_df.shape[0]

103

### Part Two - Find the Latitude and Longitude of each postal code

###### Before finding latitude and longitude, reset the index of dataframe

In [9]:
wp_df.reset_index(drop=True,inplace=True)
wp_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


###### Find latitude and longitude of each postal code

In [10]:
Latitude=[];
Longitude=[];
for i in range(0,wp_df.shape[0]):
    location=None
    geolocator = pgeocode.Nominatim('ca')
    location = geolocator.query_postal_code(wp_df.iloc[i]['Postal Code'])
    Latitude.append(location.latitude)
    Longitude.append(location.longitude)
wp_df['Latitude']=Latitude
wp_df['Longitude']=Longitude
wp_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


### Part Three - Explore and cluster the neighborhoods in Toronto

###### Filter the borough which contains only Toronto

In [11]:
index_toronto=wp_df[wp_df['Borough'].str.contains('Toronto')].index
df_new=wp_df[wp_df.index.isin(index_toronto)]
df_new.reset_index(drop=True,inplace=True)
df_new.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


###### Update URL string

In [12]:
CLIENT_ID = 'JUJNYLJMT5FGA0KG0TJT4BJ0PFVI3L3DJ3POUJPK5KDII1RP'
CLIENT_SECRET = 'JM2D1DRIVNOLXPWOZYJSI5YIDKTYCV55SQR31HLHBS1YVZNG'
VERSION = '20180605'
limit = 10
radius = 500
neighborhood_latitude=df_new.iloc[0]['Latitude']
neighborhood_longitude=df_new.iloc[0]['Longitude']
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,neighborhood_latitude,neighborhood_longitude,radius,limit)

###### Get the JSON file

In [13]:
import requests
results=requests.get(url).json()

###### Define the function to get the category name

In [14]:
def get_category(data):
    try:
        category_type=data['categories']
    except:
        category_type=data['venue.categories']
    if len(category_type)==0:
        return None
    else:
        return category_type[0]['name']

###### Get the category name of each venue

In [15]:
from pandas.io.json import json_normalize
venues=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)
#type(nearby_venues)
nearby_venues=nearby_venues.loc[:,['venue.name','venue.categories','venue.location.lat','venue.location.lng']]
nearby_venues['venue.categories']=nearby_venues.apply(get_category,axis=1)
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]
#print(col1)
nearby_venues.head(5)

C:\Users\Saikat\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,The Yoga Lounge,Yoga Studio,43.655515,-79.364955
4,Body Blitz Spa East,Spa,43.654735,-79.359874


###### Get the top 100 venue of each neighborhood within the range of 500

In [16]:
def get_nearby_venues(names,latitudes,longitudes):
    radius=500
    limit=100
    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        url="https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,limit)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [17]:
Toronto_explore=get_nearby_venues(names=df_new['Neighbourhood'],latitudes=df_new['Latitude'],longitudes=df_new['Longitude'])

In [18]:
Toronto_explore.shape

(1546, 7)

In [19]:
Toronto_explore.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa


###### onehot coding of venue category

In [20]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
Toronto_onehot=pd.get_dummies(Toronto_explore[['Venue Category']],prefix='',prefix_sep='')
first_column=Toronto_onehot.pop('Neighborhood')
Toronto_onehot.insert(loc=0,column='Neighborhood',value=Toronto_explore['Neighborhood'])
Toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

###### Group the neighborhood and find the mean of venue category

In [21]:
Toronto_grouped=Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.0,0.011111,0.022222,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.011111,0.044444,0.000000,0.000000,0.0,0.011111,0.000000,0.033333,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.022222,0.000000,0.000000,0.000000,0.0,0.0,0.044444,0.0,0.0,0.000000,0.022222,0.0,0.000000,0.000000,0.022222,0.100000,0.0,0.0,0.000000,0.0,0.0,0.0,0.011111,0.000000,0.011111,0.0,0.0,0.011111,0.0,0.0,0.0,0.000000,0.022222,0.011111,0.000000,0.0,0.011111,0.000000,0.0,0.0,0.0,0.000000,0.0,0.011111,0.000000,0.0,0.0,0.0,0.0,0.022222,0.011111,0.0,0.011111,0.0,0.0,0.0,0.011111,0.011111,0.011111,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.011111,0.0,0.0,0.0,0.000000,0.0,0.011111,0.011111,0.011111,0.011111,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.055556,0.011111,0.0,0.0,0.0,0.0,0.0,0.0,0.022222,0.033333,0.011111,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0

###### Get the top 10 venue category of each neighborhood

In [22]:
def return_top_venues(df,top):
    df_top=df.iloc[1:]
    df_top=df_top.sort_values(ascending=True)
    return df_top.index.values[0:top]

In [23]:
top_venues=10
indicators=['st','nd','rd']
columns=['Neighborhood']
for i in range(0,top_venues):
    try:
        columns.append('{}{} venue'.format(i+1,indicators[i]))
    except:
        columns.append('{}th venue'.format(i+1))
Toronto_nei_sorted=pd.DataFrame(columns=columns)
Toronto_nei_sorted['Neighborhood']=Toronto_grouped['Neighborhood']
for i in range(0,Toronto_grouped.shape[0]):
    Toronto_nei_sorted.iloc[i,1:]=return_top_venues(Toronto_grouped.iloc[i,:],top_venues)

In [24]:
Toronto_nei_sorted.head()

,Neighborhood,1st venue,2nd venue,3rd venue,4th venue,5th venue,6th venue,7th venue,8th venue,9th venue,10th venue
0,Berczy Park,Accessories Store,Light Rail Station,Lingerie Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",History Museum,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark
2,"Business reply mail Processing Centre, South C...",Accessories Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Accessories Store,Lingerie Store,Liquor Store,Malay Restaurant,Martial Arts School,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
4,Central Bay Street,Accessories Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store


###### KMeans clustering

In [25]:
kclusters=5
Toronto_cluster=Toronto_grouped.drop('Neighborhood',1)
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(Toronto_cluster)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 3, 0, 4, 4,
       0, 0, 0, 0, 4, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

###### Merging all information together

In [27]:
df_new.columns=['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
Toronto_nei_sorted.insert(0,'Cluster Labels',kmeans.labels_)
Toronto_merged=df_new
Toronto_merged=Toronto_merged.join(Toronto_nei_sorted.set_index('Neighborhood'),on='Neighborhood')
Toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st venue,2nd venue,3rd venue,4th venue,5th venue,6th venue,7th venue,8th venue,9th venue,10th venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,0,Accessories Store,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,0,Accessories Store,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,0,Accessories Store,Intersection,Irish Pub,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Lawyer,Library
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,0,Accessories Store,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,0,Accessories Store,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark


###### Toronto Map after clustering the neighborhood

In [32]:
latitude=df_new.iloc[0]['Latitude']
longitude=df_new.iloc[0]['Longitude']
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=12)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
for lat,lng,poi,cluster in zip(Toronto_merged['Latitude'],Toronto_merged['Longitude'],Toronto_merged['Neighborhood'],Toronto_merged['Cluster Labels']):
    label=folium.Popup(str(poi)+' cluster '+str(cluster),parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters